In [1]:
import pandas as pd
import os
from chunking import ParentChildChunking
from embedding import extract_table_token, bm25_embedding, openai_embeddings
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType, connections
from pymilvus import connections, utility

/Users/yj/.pyenv/versions/3.9.11/envs/myMechanic_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# NAMESPACE = 'IONIQ5_2024'
# NAMESPACE = 'SANTAFE_MX5_2023'
NAMESPACE = 'SONATA_DN8_2024'


context_file_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'
bm25_model_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/vector_db/bm25'
context_text_dir = context_file_root_path + f'/text/{NAMESPACE}'

MILVUS_HOST = os.environ['MILVUS_HOST']#'localhost' #
MILVUS_PORT = os.environ['MILVUS_PORT']
COLLECTION_NAME = "HYNDAI_CAR_MANUAL"#""

os.makedirs(bm25_model_root_path, exist_ok=True)

# connections.connect(alias="default", host=MILVUS_HOST, port=MILVUS_PORT)


In [20]:

df = pd.read_parquet(f'{context_text_dir}/doc_{NAMESPACE}_result.parquet' )

In [4]:

pcc = ParentChildChunking()
semantic_df = pcc.colelct_context_by_semantic(df)
pc_df = pcc.parent_child_chunking(semantic_df)
pc_df = extract_table_token(pc_df)

In [30]:
bm25_model_path = bm25_model_root_path + f'/bm25_{NAMESPACE}_params.json'
bm25_embeddings = bm25_embedding(pc_df['embedding_contents'].values.tolist(), bm25_model_path, train=False)
pc_df['bm25_embeddings'] = bm25_embeddings

In [ ]:
oai_embeddings = openai_embeddings(pc_df['embedding_contents'].values)
pc_df['embeddings'] = oai_embeddings

100%|██████████| 583/583 [04:44<00:00,  2.05it/s]


In [ ]:
pc_df['car_type'] = NAMESPACE
pc_df = pc_df.reset_index(drop=True).reset_index()

In [ ]:
pc_df.drop('bm25_embeddings', axis=1).to_parquet(context_file_root_path+f'/embedding/{NAMESPACE}_embeddings.parquet')

In [29]:
pc_df = pd.read_parquet(context_file_root_path+f'/embedding/{NAMESPACE}_embeddings.parquet')

In [31]:
pc_df.head()

,index,doc_id,h1,h2,chunk_group2,doc_contents,img_urls,table_contents,table_img_urls,table_csv_urls,contents_size,h3,child_chunk,table_token,embedding_contents,embeddings,car_type,bm25_embeddings
0,0,0,1. 안내 및 차량 정보,에어컨 시스템,0,"\n에어컨 시스템\n\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지...",[],[| | 종류 | 용량 | 사양 |\n|---...,[image/SONATA_DN8_2024/20_1_table.jpeg],[csv/SONATA_DN8_2024/21_1_table.csv],93,,"에어컨 시스템\n\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...","종류,용량,R,냉동유,참조,에어컨,이미지,yf,냉매,g,사양","에어컨 시스템\n\n\n에어컨 시스템\n\n에어컨 관련 작업 시, 반드시 지정된 냉...","[0.015647629275918007, -0.004448536783456802, ...",SONATA_DN8_2024,"{0: 1.4818144796619401, 1: 1.4818144796619401,..."
1,1,1,1. 안내 및 차량 정보,엔진 번호,0,\n엔진 번호\n\nSmartstream G2.5 T-GDI\n\nSmartstre...,"[image/SONATA_DN8_2024/25_2_img.png, image/SON...",[],[],[],130,,엔진 번호\n\nSmartstream G2.5 T-GDI\n\nSmartstream...,,엔진 번호\n\n\n엔진 번호\n\nSmartstream G2.5 T-GDI\n\n...,"[0.02475030906498432, -0.0011217625578865409, ...",SONATA_DN8_2024,"{20: 1.6328552529591058, 21: 1.632855252959105..."
2,2,2,1. 안내 및 차량 정보,엔진룸,0,\n엔진룸\n\nSmartstream G2.5 T-GDI\n\n이 그림은 실제 차량...,"[image/SONATA_DN8_2024/14_2_img.png, image/SON...",[],[],[],193,,엔진룸\n\nSmartstream G2.5 T-GDI\n\n이 그림은 실제 차량과 ...,,엔진룸\n\n\n엔진룸\n\nSmartstream G2.5 T-GDI\n\n이 그림...,"[0.019963478669524193, -0.02077077515423298, -...",SONATA_DN8_2024,"{13: 1.6328552529591058, 20: 1.632855252959105..."
3,3,3,1. 안내 및 차량 정보,오픈소스 소프트웨어 정보,0,\n오픈소스 소프트웨어 정보\n\n본 차량은 오픈소스 라이선스가 적용된 소프트웨어를...,[],[],[],[],283,,오픈소스 소프트웨어 정보\n\n본 차량은 오픈소스 라이선스가 적용된 소프트웨어를 포...,,오픈소스 소프트웨어 정보\n\n\n오픈소스 소프트웨어 정보\n\n본 차량은 오픈소스...,"[-0.021060850471258163, 0.004754357505589724, ...",SONATA_DN8_2024,"{13: 1.3227518223363675, 33: 1.322751822336367..."
4,4,4,1. 안내 및 차량 정보,자기 인증 라벨,0,\n자기 인증 라벨\n\n자기 인증 라벨은 차량이 대한민국 자동차 관리법령에 맞게 ...,[image/SONATA_DN8_2024/24_4_img.png],[],[],[],56,,자기 인증 라벨\n\n자기 인증 라벨은 차량이 대한민국 자동차 관리법령에 맞게 제작...,,자기 인증 라벨\n\n\n자기 인증 라벨\n\n자기 인증 라벨은 차량이 대한민국 자...,"[-0.001701890490949154, -0.04354502260684967, ...",SONATA_DN8_2024,"{33: 1.6002330270094294, 65: 1.600233027009429..."


In [ ]:
pc_df['img_urls'] = pc_df['img_urls'].apply(lambda img_urls: list(map(lambda img_url: '/'.join(img_url.split('/')[-3:]), img_urls)))
pc_df['table_img_urls'] = pc_df['table_img_urls'].apply(lambda img_urls: list(map(lambda img_url: '/'.join(img_url.split('/')[-3:]), img_urls)))
pc_df['table_csv_urls'] = pc_df['table_csv_urls'].apply(lambda img_urls: list(map(lambda img_url: '/'.join(img_url.split('/')[-3:]), img_urls)))

In [32]:
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False)
car_type_filed = FieldSchema(name="car_type", dtype=DataType.VARCHAR, max_length=100, is_primary=False, auto_id=False,
                           description='차종')
major_h_filed = FieldSchema(name="majorheading", dtype=DataType.VARCHAR, max_length=100, is_primary=False, auto_id=False,
                           description='설명서의 대제목')
minor_h_filed = FieldSchema(name="minorheading", dtype=DataType.VARCHAR, max_length=100, is_primary=False, auto_id=False,
                           description='설명서의 소제목')
minor_sub_id_filed = FieldSchema(name="minorheading_sub_id", dtype=DataType.INT64, max_length=100, is_primary=False, auto_id=False,
                            description='소제목 ID')

parent_doc_id_filed = FieldSchema(name="parent_doc_id", dtype=DataType.INT64, max_length=1000, is_primary=False, auto_id=False,
                            description='Parent 문서ID ')

contents_filed = FieldSchema(name="doc_contents", dtype=DataType.VARCHAR, max_length=40000, is_primary=False, auto_id=False,
                            description='설명서의 내용')
imgurl_filed = FieldSchema(name="img_urls", dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=20, max_length=50,
                            description='이미지 URL 경로')
tblimgurl_field = FieldSchema(name="table_img_urls", dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=20, max_length=50,
                            description='표 이미지 URL 경로')
tblcsvurl_filed = FieldSchema(name="table_csv_urls", dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=20, max_length=3000,
                            description='표 csv 경로')

embedding_contents_filed = FieldSchema(name="embedding_contents", dtype=DataType.VARCHAR, max_length=10000, is_primary=False, auto_id=False,
                            description='검증용 임베딩 내용')

vector_filed = FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=3072)
bm_25_vector_filed = FieldSchema(name="bm25_vector", dtype=DataType.SPARSE_FLOAT_VECTOR)
# 컬렉션 스키마 정의
schema = CollectionSchema(
    fields=[
        id_field, 
        car_type_filed,
        major_h_filed,
        minor_h_filed,
        minor_sub_id_filed,
        parent_doc_id_filed,
        contents_filed,
        imgurl_filed,
        tblimgurl_field,
        tblcsvurl_filed,
        embedding_contents_filed,
        vector_filed,
        bm_25_vector_filed
        ],
    description="현대 기아 차량 매뉴얼",
    partition_key_field="car_type"

)

# 컬렉션 생성
if COLLECTION_NAME not in utility.list_collections():
    collection = Collection(name=COLLECTION_NAME, schema=schema)
else:
    collection = Collection(name=COLLECTION_NAME)



In [10]:
# utility.drop_collection(COLLECTION_NAME)

In [33]:
#Index 생성
# Pandas DataFrame에서 데이터 추출 및 Milvus에 삽입
ids = pc_df['index'].to_list()
car_types = pc_df['car_type'].to_list()
h1s = pc_df['h1'].to_list()
h2s = pc_df['h2'].fillna('').astype(str).to_list()
h3s = pc_df['chunk_group2'].to_list()
parent_doc_id= pc_df['doc_id'].to_list()

cons = pc_df['doc_contents'].to_list()
img_urls = pc_df['img_urls'].to_list()
table_img_urls = pc_df['table_img_urls'].to_list()
table_csv_urls = pc_df['table_csv_urls'].to_list()
embedding_cont = pc_df['embedding_contents'].to_list()

vectors = pc_df['embeddings'].to_list()
bm25_vectors = pc_df['bm25_embeddings'].to_list()

In [34]:
NAMESPACE

'SONATA_DN8_2024'

In [ ]:
# expr = f'''car_type=="{NAMESPACE}"'''
# collection.delete(expr)

(insert count: 0, delete count: 36, upsert count: 0, timestamp: 0, success count: 0, err count: 0)

In [17]:
expr = f'''car_type=="{NAMESPACE}"'''
response = collection.query(expr, output_fields=["car_type", "doc_contents"])

In [18]:
len(response)

619

In [35]:
mr = collection.insert([ids, car_types, h1s, h2s, h3s, parent_doc_id, cons, img_urls, table_img_urls, table_csv_urls, embedding_cont, vectors, bm25_vectors])


In [16]:
M = 20
efConstruction=100

In [17]:
collection.create_index(
    field_name="vector", 
    index_params={
        "index_type": "HNSW", 
        "index_name":"ann_index",
        "metric_type": "COSINE", 
        "params": {"M": M, 'efConstruction':efConstruction} # Cluster 개수
        }
)

Status(code=0, message=)

In [18]:

collection.create_index(
    field_name= "bm25_vector",
    index_params={
        "index_name": "bm25_index",
        "index_type": "SPARSE_INVERTED_INDEX", # the type of index to be created. set to `SPARSE_INVERTED_INDEX` or `SPARSE_WAND`.
        "metric_type": "IP", # the metric type to be used for the index. Currently, only `IP` (Inner Product) is supported.
        "params": {"drop_ratio_build": 0.01}, # the ratio of small vector values to be dropped during indexing.
})

Status(code=0, message=)

In [ ]:
COLLECTION_NAME

'HYNDAI_CAR_MANUAL'

In [14]:
collection = Collection(COLLECTION_NAME)
collection.load()